In [1]:
import numpy as np
import pandas as pd
import torch as torch

In [2]:
dataset=pd.read_csv('Z:/users/songyeon/DrugTargetIdentification/PRISM(19Q4)/secondary-screen-replicate-treatment-info.csv', low_memory=False)

In [3]:
dataset.head(5)

,column_name,broad_id,dose,perturbation_type,screen_id,detection_plate,compound_plate,well,name,moa,target,disease.area,indication,smiles,phase
0,PROS001_PR500_120H_X1_P4:A01::MTS010,BRD-K36788280-001-01-2,0.15625,experimental_treatment,MTS010,PROS001_PR500_120H_X1_P4,PROS001_PR500,A01,ribociclib,CDK inhibitor,"CDK4, CDK6",oncology,breast cancer,CN(C)C(=O)c1cc2cnc(Nc3ccc(cn3)N3CCNCC3)nc2n1C1...,Launched
1,PROS001_PR500_120H_X1_P4:A02::MTS010,BRD-K88510285-001-50-9,20.00070,positive_control,MTS010,PROS001_PR500_120H_X1_P4,PROS001_PR500,A02,bortezomib,"NFkB pathway inhibitor, proteasome inhibitor","PSMA1, PSMA2, PSMA3, PSMA4, PSMA5, PSMA6, PSMA...",hematologic malignancy,"multiple myeloma, mantle cell lymphoma (MCL)",CC(C)C[C@H](NC(=O)[C@H](Cc1ccccc1)NC(=O)c1cncc...,Launched
2,PROS001_PR500_120H_X1_P4:A04::MTS010,BRD-K99749624-001-07-0,0.03910,experimental_treatment,MTS010,PROS001_PR500_120H_X1_P4,PROS001_PR500,A04,linifanib,"PDGFR tyrosine kinase receptor inhibitor, VEGF...","CSF1, CSF1R, FLT1, FLT3, FLT4, KDR, KIT, PDGFR...",NaN,NaN,Cc1ccc(F)c(NC(=O)Nc2ccc(cc2)-c2cccc3[nH]nc(N)c...,Phase 3
3,PROS001_PR500_120H_X1_P4:A05::MTS010,BRD-K85402309-043-01-9,0.00061,experimental_treatment,MTS010,PROS001_PR500_120H_X1_P4,PROS001_PR500,A05,dovitinib,"EGFR inhibitor, FGFR inhibitor, FLT3 inhibitor...","CSF1R, EGFR, FGFR1, FGFR2, FGFR3, FLT1, FLT3, ...",NaN,NaN,CN1CCN(CC1)c1ccc2nc([nH]c2c1)-c1c(N)c2c(F)cccc...,Phase 3
4,PROS001_PR500_120H_X1_P4:A06::MTS010,BRD-K50168500-001-07-9,10.00000,experimental_treatment,MTS010,PROS001_PR500_120H_X1_P4,PROS001_PR500,A06,canertinib,EGFR inhibitor,"AKT1, EGFR, ERBB2, ERBB4",NaN,NaN,Fc1ccc(Nc2ncnc3cc(OCCCN4CCOCC4)c(NC(=O)C=C)cc2...,Phase 3


In [4]:
subset = dataset[['name', 'target']]

In [5]:
subset.head(11)

,name,target
0,ribociclib,"CDK4, CDK6"
1,bortezomib,"PSMA1, PSMA2, PSMA3, PSMA4, PSMA5, PSMA6, PSMA..."
2,linifanib,"CSF1, CSF1R, FLT1, FLT3, FLT4, KDR, KIT, PDGFR..."
3,dovitinib,"CSF1R, EGFR, FGFR1, FGFR2, FGFR3, FLT1, FLT3, ..."
4,canertinib,"AKT1, EGFR, ERBB2, ERBB4"
5,ispinesib,KIF11
6,ispinesib,KIF11
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [6]:
subset = subset[subset['name'].notnull()] #Drug 정보가 없는 (name=NaN) 경우 제외

In [7]:
subset=subset.fillna("NA") #target이 없는 (NaN)인 경우 NA로 결측값 변경

In [8]:
subset["target"] = subset["target"].map(lambda a: a.split(",")) #구분자로 분리해서 리스트 생성 (도쿠위키 참고)

In [9]:
subset = pd.DataFrame.explode(subset, "target") #행별로 나눔

In [10]:
subset

,name,target
0,ribociclib,CDK4
0,ribociclib,CDK6
1,bortezomib,PSMA1
1,bortezomib,PSMA2
1,bortezomib,PSMA3
...,...,...
48966,vinblastine,TUBA1A
48966,vinblastine,TUBB
48966,vinblastine,TUBD1
48966,vinblastine,TUBE1


In [11]:
subset=subset.drop_duplicates() #중복값 제거. 중복값이 생기는 이유는 약물 농도를 4배씩 줄여가며 8번 진행했고, 실험을 3번 돌렸기 때문에 24개의 같은 값이 생김

In [12]:
subset

,name,target
0,ribociclib,CDK4
0,ribociclib,CDK6
1,bortezomib,PSMA1
1,bortezomib,PSMA2
1,bortezomib,PSMA3
...,...,...
48943,vinblastine,TUBA1A
48943,vinblastine,TUBB
48943,vinblastine,TUBD1
48943,vinblastine,TUBE1


In [13]:
subset=pd.crosstab(subset.name, subset.target) #cross tabulation 생성

In [14]:
subset

target,ABL2,ACACB,ACVR1,ADAM28,ADAMTS5,ADCY5,ADORA2A,ADORA2B,ADORA3,ADRA1B,...,USP14,USP7,UTS2R,VCP,VDAC2,VDR,VKORC1,WEE1,XIAP,XPO1
name,,,,,,,,,,,,,,,,,,,,,
1-azakenpaullone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1-naphthyl-PP1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1-phenylbiguanide,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10-deacetylbaccatin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10-hydroxycamptothecin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zaleplon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zardaverine,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ziprasidone,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
subset.to_csv("Z:/users/songyeon/DrugTargetIdentification/PRISM(19Q4)/secondary-screen-replicate-treatment-info_DrugTargetRelationMatrix.csv", header=True)